# Generating CESM Grid, Domain, and Mapping files for MOM6

This notebook describes how to generate SCRIP file, binary grid files (for CICE), and domain and mapping files (for CIME) to run MOM6 within CESM framework. Overview of the workflow for generating these files is as follows:

1. Begin with an existing netcdf grid file.
2. Generate the SCRIP grid file. (to be used when creating mapping files)
3. Generate the binary grid files. (to be used by CICE)
4. Generate the mapping and domain files. (to be used by CIME)
5. Configure CIME
6. Create a new case with the new resolution
7. Prepare MOM6 input files
8. Copy SourceMods and user_nl files

Steps 1 through 5 needs to be done only once for a new grid, while steps 6, 7, and 8 need to be repeated for each new case.

## 1. Begin with an existing netcdf grid file.

First, a netcdf grid file containing the grid variables, e.g., coordinates, areas, masks, etc. is required. (This file is NOT the SCRIP grid file or MOM6 supergrid file). If you don't have the netcdf grid file ready, follow the steps described in the notebook "gen_nc_grid.ipynb" to generate one.The header of an example netcdf grid file is as follows:

See [Appendix-A](#Appendix---A:-Example-netcdf-grid-file-header) for an example netcdf grid file header


## 2. Generate the SCRIP grid file

### 2.1 Global Ocean SCRIP file

Given the NetCDF grid file and a rectilinear togopraphy file, e.g., ETOPO1, generate the SCRIP file. This is done by running the following NCL script, which is composed of three main parts:

1. Convert the original source topography grid to a SCRIP convention file.
2. Convert the netcdf grid file to a SCRIP convention file.
3. Generate the weights that map from the original topography grid to the ocean grid to be generated.

See NCL script: [gen_scrip.ncl](#B.1:-gen_scrip.ncl)

### 2.2 Coastal SCRIP file

The coastal SCRIP file is used as an input when generating the rof->ocn map. To generate this file, customize and run the following NCL script.

See NCL script: [gen_coastal_scrip.ncl](#B.2:-gen_coastal_scrip.ncl)


## 3. Generate the binary grid files

CICE requires two binary grid files, horizontal grid file and topography file, which should be in the same format as the corresponding POP input files. Generate these files by running the following ncl script:

**Note: Update the file paths following the ```begin``` keyword in the script accordingly.**

See NCL script: [gen_cice_gridfiles.ncl](#B.3:-gen_cice_gridfiles.ncl)

## 4. Generate the mapping and domain files. 

CESM comes with a suite of auxiliary tools including scripts to generate domain and mapping files for its components. Usage of these tools are explained in this section. 

Note: ```$SRCROOT``` in the following commands denotes the full pathname of CESM source root directory.

### 4.1 Generate atm<->ocn mapping files

- First, navigate to check_maps directory to build the tool that checks the quality of the maps after they are generated.
```
cd $SRCROOT/cime/tools/mapping/check_maps/
```

- Follow the instructions in ```README``` file to build the check_maps tool.

- Navigate to ```gen_mapping_files``` directory and run ```gen_cesm_maps.sh``` script to generate atm<->ocn mapping files as follows.
```
cd $SRCROOT/cime/tools/mapping/gen_mapping_files/
./gen_cesm_maps.sh -fatm ATM_SCRIP_FILE -natm ATM_GRID_NAME -focn OCN_SCRIP_FILE -nocn OCN_GRID_NAME
```
where ```ATM_SCRIP_FILE``` is the path to the atmosphere SCRIP grid file (e.g., T62 grid on cheyenne: ```/glade/p/cesmdata/cseg/mapping/grids/T62_040121.nc```), ```ATM_GRID_NAME``` is the name of the atmosphere grid, e.g., T62, ```OCN_SCRIP_FILE``` is the ocean SCRIP grid file path that you created in Step 2, and ```OCN_GRID_NAME``` is the name of the ocean grid, e.g., tx0.66v1.

The tool generates three atm->ocn and two ocn->atm maps. These will be described in later sections.

### 4.2 Generate rof->ocn mapping files

- Navigate to runoff_to_ocn directory:
```
cd $SRCROOT/cime/tools/mapping/gen_mapping_files/runoff_to_ocn/
```

- Follow the instructions in ```INSTALL``` file to install the tool.
- Create a directory where you want to generate the rof2ocn mapping file. In the following commands ```$ROF2OCN``` denotes this directory.
- Create a namelist file inside ```$ROF2OCN``` directory. The namelist entries to be provided in this file are described in ```$SRCROOT/cime/tools/mapping/gen_mapping_files/runoff_to_ocn/README``` file. An example file (for rx1 runoff grid to tx0.66v1 ocn grid) is as follows:
```
&input_nml
   gridtype     = 'obs'
   file_roff    = '/glade/p/cesm/cseg/inputdata/lnd/dlnd7/RX1/runoff.daitren.annual.090225.nc'
   file_ocn     = '/glade/p/work/altuntas/cesm.input/grid/scrip/tx0.66v1_SCRIP_171107.nc'
   file_ocn_coastal_mask = '/glade/p/work/altuntas/mom.input/tx0.66v1/gen_grid_171220/tx0.66v1_SCRIP_coastal_171220.nc'
   file_nn      = 'map_rx1_to_tx0.66v1_nn_171226.nc'
   file_smooth  = 'map_rx1_to_tx0.66v1_sm_e1000r300_171226.nc'
   file_new     = 'map_rx1_to_tx0.66v1_nnsm_e1000r300_171226.nc'
   title        = 'runoff map: rx1 -> rx0.66v1, nearest neighbor and smoothed '
   eFold        = 1000000.0
   rMax         =  300000.0
   step1 = .true.
   step2 = .true.
   step3 = .true.
/
```

- Run ```runoff_map``` tool as follows:
```
$SRCROOT/cime/tools/mapping/gen_mapping_files/runoff_to_ocn/runoff_map < NML_FILE_PATH
```
where ```NML_FILE_PATH``` is the path to the namelist file.
The tool generates three mapping files in three steps, as described in the README file. In MOM6, we only use the map that is generated in the final step, i.e., whose path is specified in ```file_new``` entry in the namelist file. 

### 4.3 Generate the domain files

Domain files, read by CIME, contain mask information about the grids to be mapped. These files are similarly generated using the CIME tool, called gen_domain_files. 


- First, navigate to gen_domain_files directory to build the tool:
```
cd $SRCROOT/cime/tools/mapping/gen_domain_files/
```

- Follow the instructions in ```INSTALL``` file to build the gen_domain_files tool.

- To generate the domain files, follow the instructions in ```README``` file. The following is an example usage where the domain files (three in total) for T62_t061 resolution are generated.

$SRCROOT/cime/tools/mapping/gen_domain_files/gen_domain -m /glade/p/work/altuntas/cesm.input/map/T62_tx0.66v1/map_tx0.66v1_TO_T62_aave.171017.nc -o tx0.66v1 -l T62


## 5. Configure CIME

The new grid needs to be added to CIME. To do this, the resolution alias (e.g., T62_t061), and the paths to domain and mapping files (generated in step 4) should be added to config_grids.xml file in cime/config/cesm/ directory as follows.
 
#### Note: See the following github commit for an example of how a grid gets added to CIME: https://github.com/ESMCI/cime/commit/1879acb7b07cd13f64d237f9573a6ca8b902f997
 
- Add the new grid alias, e.g.,
 ```
    <model_grid alias="T62_t061" not_compset="_CAM">
      <grid name="atm">T62</grid>
      <grid name="lnd">T62</grid>
      <grid name="ocnice">tx0.66v1</grid>
    </model_grid>
```

- Add the atm|lnd maks file directory, e.g.:
```
<file grid="atm|lnd" mask="tx0.66v1">/glade/p/work/altuntas/cesm.input/domain/T62_tx0.66v1/domain.lnd.T62_tx0.66v1.171017.nc</file>
```

- Add the ocn mask file directory, e.g.:
```
<file grid="ocnice" mask="tx0.66v1">/glade/p/work/altuntas/cesm.input/domain/T62_tx0.66v1/domain.ocn.T62_tx0.66v1.171017.nc</file>
```

- Define the new domain, e.g.,
```
<domain name="tx0.66v1">
  <nx>540</nx> <ny>458</ny>
  <file grid="ocnice">/glade/p/work/altuntas/cesm.input/domain/T62_tx0.66v1/domain.ocn.tx0.66v1.171017.nc</file>
  <desc>tx0.66v1 is tripole v1 0.66-deg MOM6 grid:</desc>
  <support>Experimental for MOM6 experiments</support>
</domain>
```

- Add atm<->ocn mapping file directories:
```
<gridmap atm_grid="T62" ocn_grid="tx0.66v1">
 <map name="ATM2OCN_FMAPNAME">/glade/p/work/altuntas/cesm.input/map/T62_tx0.66v1/map_T62_TO_tx0.66v1_aave.171017.nc</map>
 <map name="ATM2OCN_SMAPNAME">/glade/p/work/altuntas/cesm.input/map/T62_tx0.66v1/map_T62_TO_tx0.66v1_blin.171017.nc</map>
 <map name="ATM2OCN_VMAPNAME">/glade/p/work/altuntas/cesm.input/map/T62_tx0.66v1/map_T62_TO_tx0.66v1_blin.171017.nc</map>
 <map name="OCN2ATM_FMAPNAME">/glade/p/work/altuntas/cesm.input/map/T62_tx0.66v1/map_tx0.66v1_TO_T62_aave.171017.nc</map>
 <map name="OCN2ATM_SMAPNAME">/glade/p/work/altuntas/cesm.input/map/T62_tx0.66v1/map_tx0.66v1_TO_T62_aave.171017.nc</map>
</gridmap
```

- Add rof->ocn mapping file directories:
```
<gridmap rof_grid="rx1" ocn_grid="tx0.66v1" >
   <map name="ROF2OCN_LIQ_RMAPNAME">/glade/p/work/altuntas/cesm.input/map/rx1_tx0.66v1/map_rx1_to_tx0.66v1_nnsm_e1000r300_171010.nc</map>
   <map name="ROF2OCN_ICE_RMAPNAME">/glade/p/work/altuntas/cesm.input/map/rx1_tx0.66v1/map_rx1_to_tx0.66v1_nnsm_e1000r300_171010.nc</map>
</gridmap>
```


#### Note: the steps above need to be done only once for a new grid. The steps below need to be repeated for each new case.

## 6. Create a new case with the new resolution

- Create the new case, e.g.:
```
create_newcase --case gmom.e20.GMOM.T62_t061.001 --compset 2000_DATM%NYF_SLND_CICE_MOM6_DROF%NYF_SGLC_SWAV --res T62_t061 --user-compset --run-unsupported
```
- Run ./case.setup


## 7. Prepare MOM6 input files

Collect the following set of input files and copy them into RUNDIR directory of the case:
- diag_table
- input.nml
- MOM_input
- MOM_override
- INPUT/ocean_hgrid.nc
- INPUT/ocean_topog.nc
- INPUT/ocean_vgrid.nc
- WOA05_pottemp_salt.nc

Make sure that ocean_hgrid.nc, ocean_topog.nc, and ocean_vgrid.nc files are compatible with the MOM6 supergrid file (and so all other files generated by following this notebook.)

## 8. Copy SourceMods:

### 8.1 CIME SourceMod:

At the initialization of all case runs, CIME checks for inconsistencies between grids, mapping files, domain files, etc. When comparing ice and ocean grids, for instance, CIME checks whether the difference between the areas of overlapping ice and ocean cells are less then a predefined tolerance. For an experimental grid, this tolerance may need to be relaxed until the grid is finalized. The following SourceMod is an example of how this may be done:
```
/glade/p/work/altuntas/cesm.sourcemods/cesm2_0_beta06.mom6/src.drv/seq_domain_mct.F90
```
To run a case with T62_t061 grid, copy the file above to SourceMods/src.drv/ directory within your CASEROOT before building your case. 

### 8.2 CICE SourceMods:

#### 8.2.1 Add the new grid:

Add the new grid and the paths to its files (topography and horizontal grid) to components/cice/cime_config/namelist_definition_cice.xml file. An example SourceMod can be found in:
```
/glade/p/work/altuntas/cesm.sourcemods/cesm2_0_beta06.mom6/src.cice/namelist_definition_cice.xml
```
To run a case with T62_t061 grid, copy the file above to SourceMods/src.cice/ directory within your CASEROOT before building your case. 

#### 8.2.2 Fix the cell coordinates interval:

CICE assumes that the coordinates of cells read from the horizontal grid file are confined to be between -180 and 180 degrees (because of the way the intrinsic function atan2 works). If this is not the case, (like tx0.66v1), copy the following SourceMod to fix this issue:
```
/glade/p/work/altuntas/cesm.sourcemods/cesm2_0_beta06.mom6/src.cice/ice_grid.F90
```
To run a case with T62_t061 grid, copy the file above to SourceMods/src.cice/ directory within your CASEROOT before building your case. 

### 8.2 CICE user_nl_cice changes:

If there is not an initial conditions file availabe for CICE yet, paste the following line to the end of user_nl_cice file to experiment with the new grid using the default initial conditions for CICE:
```
ice_ic = 'UNSET'
```

---------
## Final Steps:

After copying the SourceMods and user_nl files as described, build and submit the case.

----------

## Appendix - A: Example NetCDF grid file header

```
netcdf tx0.66v1_grid {
dimensions:
	nyp = 459 ;
	nxp = 541 ;
	ny = 458 ;
	nx = 540 ;
variables:
	double tlon(ny, nx) ;
		tlon:long_name = "array of t-grid longitudes" ;
		tlon:units = "degrees_east" ;
	double tlat(ny, nx) ;
		tlat:long_name = "array of t-grid latitudes" ;
		tlat:units = "degrees_north" ;
	double ulon(ny, nxp) ;
		ulon:long_name = "array of u-grid longitudes" ;
		ulon:units = "degrees_east" ;
	double ulat(ny, nxp) ;
		ulat:long_name = "array of u-grid latitudes" ;
		ulat:units = "degrees_north" ;
	double vlon(nyp, nx) ;
		vlon:long_name = "array of v-grid longitudes" ;
		vlon:units = "degrees_east" ;
	double vlat(nyp, nx) ;
		vlat:long_name = "array of v-grid latitudes" ;
		vlat:units = "degrees_north" ;
	double qlon(nyp, nxp) ;
		qlon:long_name = "array of q-grid longitudes" ;
		qlon:units = "degrees_east" ;
	double qlat(nyp, nxp) ;
		qlat:long_name = "array of q-grid latitudes" ;
		qlat:units = "degrees_north" ;
	double dxt(ny, nx) ;
		dxt:long_name = "x-distance between u-points, centered at t" ;
		dxt:units = "meters" ;
	double dyt(ny, nx) ;
		dyt:long_name = "y-distance between v-points, centered at t" ;
		dyt:units = "meters" ;
	double dxCv(ny, nx) ;
		dxCv:long_name = "x-distance between  q-points, centered at v" ;
		dxCv:units = "meters" ;
	double dyCu(ny, nx) ;
		dyCu:long_name = "y-distance between  q-points, centered at u" ;
		dyCu:units = "meters" ;
	double dxCu(ny, nx) ;
		dxCu:long_name = "x-distance between  t-points, centered at u" ;
		dxCu:units = "meters" ;
	double dyCv(ny, nx) ;
		dyCv:long_name = "y-distance between  t-points, centered at v" ;
		dyCv:units = "meters" ;
	double tarea(ny, nx) ;
		tarea:long_name = "area of t-cells" ;
		tarea:units = "meters^2" ;
	double tmask(ny, nx) ;
		tmask:long_name = "ocean fraction at t-cell centers" ;
		tmask:units = "none" ;
	double angle(ny, nx) ;
		angle:long_name = "angle grid makes with latitude line" ;
		angle:units = "degrees" ;
	double depth(ny, nx) ;
		depth:long_name = "depth at h points" ;
		depth:units = "meters" ;
	double ar(ny, nx) ;
		ar:long_name = "grid aspect ratio (dyt/dxt)" ;
		ar:units = "none" ;
	double egs(ny, nx) ;
		egs:long_name = "grid effective grid spacing" ;
		egs:units = "degrees" ;

// global attributes:
		:Description = "MOM6 2/3 degree tripolar grid (ORCA type)" ;
		:Author = "Fred Castruccio (fredc@ucar.edu)" ;
		:Created = "2017-10-26T09:28:05.530173" ;
		:type = "tx0.66v1 file" ;
}
```

## Appendix - B: NCL Scripts

### B.1: gen_scrip.ncl

**Note: Update the file paths following the ```begin``` keyword in the script accordingly.**


```
; gen_scrip.ncl : 

load "$NCARG_ROOT/lib/ncarg/nclscripts/csm/gsn_code.ncl"
load "$NCARG_ROOT/lib/ncarg/nclscripts/csm/gsn_csm.ncl"
load "$NCARG_ROOT/lib/ncarg/nclscripts/csm/contributed.ncl"
load "$NCARG_ROOT/lib/ncarg/nclscripts/esmf/ESMF_regridding.ncl"
setvalues NhlGetWorkspaceObjectId()
        "wsMaximumSize" : 100000000000
end setvalues

begin
;---Input files
    topoFilePath  = "/glade/p/work/altuntas/mom.input/tx0.66v1/gen_grid_180604/ETOPO1_Bed_c_gmt4.grd"
    ncGrdFilePath = "/glade/p/work/altuntas/scripts/notebooks/gen_tx0.66v1_180604/tx0.66v1_grid.nc"
    maskFilePath  = "/glade/p/work/gmarques/cesm/grid/MOM6/tx0.66v1/tx0.66v1_ocean_topo_edited_180405.nc"

;---Output (and input) files
    srcGridPath = "/glade/p/work/altuntas/mom.input/tx0.66v1/gen_grid_180604/ETOPO1_SCRIP_180604.nc"
    dstGridPath = "/glade/p/work/altuntas/mom.input/tx0.66v1/gen_grid_180604/tx0.66v1_SCRIP_180604.nc"
    wgtFilePath = "/glade/p/work/altuntas/mom.input/tx0.66v1/gen_grid_180604/ETOPO1_2_tx0.66v1.nc"

;----------------------------------------------------------------------
; Convert original source ETOPO1 grid to a SCRIP convention file.
;----------------------------------------------------------------------

    src_file = addfile(topoFilePath,"r")
    srclon      = src_file->x(:)
    srclat      = src_file->y(:)
    srctopo     = src_file->z(:,:)

    Opt                = True
    Opt@ForceOverwrite = True
    Opt@PrintTimings   = True
    Opt@NetCDFType     = "netcdf4"
    Opt@Title          = "ETOPO1 Grid"

    rectilinear_to_SCRIP(srcGridPath,srclat,srclon,Opt)

;---Clean up
    delete(Opt)

;----------------------------------------------------------------------
; Convert destination grid to a SCRIP convention file.
;----------------------------------------------------------------------
    mom6_grd_file = addfile(ncGrdFilePath,"r")
    tlon     = mom6_grd_file->tlon(:,:)
    tlat     = mom6_grd_file->tlat(:,:)

    Opt                = True
    Opt@ForceOverwrite = True
    Opt@PrintTimings   = True
    Opt@NetCDFType     = "netcdf4"
    Opt@Title          = "tx0.66v1 Grid"


    ; Corners' lat lon --------------------------------------------------

    grid_dimsizes = getfiledimsizes(mom6_grd_file)
    nyp = grid_dimsizes(0)
    nxp = grid_dimsizes(1)
    ny = grid_dimsizes(2)
    nx = grid_dimsizes(3)
    n = nx*ny

    Opt@GridCornerLat = new(4*n,double)
    Opt@GridCornerLon = new(4*n,double)

    do j=0,ny-1
      do i=0,nx-1
        bi = (j*nx+i)*4 ; index of 1st corner in cell 

        Opt@GridCornerLat(bi)   = mom6_grd_file->qlat(j,i)
        Opt@GridCornerLat(bi+1) = mom6_grd_file->qlat(j,i+1)
        Opt@GridCornerLat(bi+2) = mom6_grd_file->qlat(j+1,i)
        Opt@GridCornerLat(bi+3) = mom6_grd_file->qlat(j+1,i+1)
        Opt@GridCornerLon(bi)   = mom6_grd_file->qlon(j,i)
        Opt@GridCornerLon(bi+1) = mom6_grd_file->qlon(j,i+1)
        Opt@GridCornerLon(bi+2) = mom6_grd_file->qlon(j+1,i)
        Opt@GridCornerLon(bi+3) = mom6_grd_file->qlon(j+1,i+1)
      end do
    end do

    ; Mask --------------------------------------------------

    maskFile = addfile(maskFilePath,"r")
    Opt@GridMask = toint(maskFile->mask)

    ; Generate the script file ------------------------------

    curvilinear_to_SCRIP(dstGridPath,tlat,tlon,Opt)

;---Clean up
    delete(Opt)

    ; Append Area to SCRIP file------------------------------

    scripFile = addfile(dstGridPath,"w")

    grid_size = dimsizes(scripFile->grid_center_lat)
    grid_area = new(grid_size,double)
    grid_area!0 = "grid_size"
  
    do i=0,grid_size-1
      temp_tlat = (/ scripFile->grid_corner_lat(i,3), \
                scripFile->grid_corner_lat(i,1), \
                scripFile->grid_corner_lat(i,0), \
                scripFile->grid_corner_lat(i,2)    /)
      temp_tlon = (/ scripFile->grid_corner_lon(i,3), \
                scripFile->grid_corner_lon(i,1), \
                scripFile->grid_corner_lon(i,0), \
                scripFile->grid_corner_lon(i,2)    /)
  
      grid_area(i) = area_poly_sphere(temp_tlat, temp_tlon, 1)
  
    end do
  
    scripFile->grid_area = grid_area


;----------------------------------------------------------------------
; Generate the weights that take you from the ETOPO1 grid to a
; tx0.66v1 degree grid.
;----------------------------------------------------------------------
    Opt                      = True
    Opt@InterpMethod         = "bilinear"     ; default
    Opt@ForceOverwrite       = True
    Opt@PrintTimings         = True
    Opt@NetCDFType           = "netcdf4"

    ESMF_regrid_gen_weights(srcGridPath,dstGridPath,wgtFilePath,Opt)

    delete(Opt)

end
```

### B.2: gen_coastal_scrip.ncl

**Note: Update the file paths and grid dimensions following the ```begin``` keyword in the script accordingly.**


```
; gen_coastal_scrip.ncl : 

begin

  ; old grid file
  scripFile = "/glade/p/work/altuntas/mom.input/tx0.66v1/gen_grid_171220/tx0.66v1_SCRIP_171220.nc"
  coastalScripFile = "/glade/p/work/altuntas/mom.input/tx0.66v1/gen_grid_171220/tx0.66v1_SCRIP_coastal_171220.nc"

  nx = 540
  ny = 458

  ; -----------------------------------------------------------------------

  inputFile = addfile(scripFile,"r")
  system("/bin/rm -f " + coastalScripFile)
  outputFile = addfile(coastalScripFile,"c")

  ; copy global file attributes:
  varatts = getvaratts(inputFile)
  if (.not. ismissing(varatts(0))) then
    fileattdef(outputFile,inputFile)
  end if

  ; copy all variables:
  varNames_in = getfilevarnames(inputFile);
  nNames = dimsizes(varNames_in)
  do i=0,nNames-1
    outputFile->$varNames_in(i)$ = inputFile->$varNames_in(i)$
  end do

  grid_size = dimsizes(inputFile->grid_center_lat)
  outputFile->grid_imask = inputFile->grid_imask

  do i=0,nx-1
    do j=0,ny-1

      c = inputFile->grid_imask(i+nx*j)                     ; center

      if (c.eq.1) then
        n = inputFile->grid_imask(i+nx*( mod(j+1, ny) ) )     ; north
        s = inputFile->grid_imask(i+nx*( mod(j-1+ny, ny) ) )  ; south
        e = inputFile->grid_imask( (mod(i-1+nx,nx) +nx*j) )   ; east
        w = inputFile->grid_imask( (mod(i+1,nx) +nx*j) )      ; west

        if (j.eq.(ny-1)) then
           n=1
        end if

        outputFile->grid_imask(i+nx*j) = 1-(n*s*e*w)

      else
        outputFile->grid_imask(i+nx*j) = c
      end if

    end do
  end do

end

```

### B.3: gen_cice_gridfiles.ncl

**Note: Update the file paths following the ```begin``` keyword in the script accordingly.**

```
; gen_cice_gridfiles.ncl

; Input/Output File Paths:
; ===================================================================

; Input Vertical Grid File:
in_vgridPath      = "/glade/p/work/altuntas/mom.input/tx0.66v1/case_input_171220/INPUT/ocean_vgrid.nc"

; Input Topography File:
in_topoPath       = "/glade/p/work/altuntas/mom.input/tx0.66v1/case_input_171220/INPUT/ocean_topog.nc"

; Input Horizontal Grid File:
in_superGridPath  = "/glade/p/work/altuntas/mom.input/tx0.66v1/gen_grid_171220/tx0.66v1_grid.nc"

; Input Mask File:
in_maskPath       = "/glade/p/work/altuntas/mom.input/tx0.66v1/gen_grid_171220/ocean_mask_edited.nc"

; Output Horizontal Grid File (binary):
out_hgridPath     = "/glade/p/work/altuntas/mom.input/tx0.66v1/cice_input_171220/horiz_grid_20171220.ieeer8"

; Output Topography file (binary):
out_topoPath      = "/glade/p/work/altuntas/mom.input/tx0.66v1/cice_input_171220/topography_20171220.ieeei4"

; ===================================================================

in_vgridFile = addfile(in_vgridPath,"r")
in_topoFile = addfile(in_topoPath,"r")
in_superGridFile = addfile(in_superGridPath,"r")

sizes = dimsizes(in_topoFile->depth)
ny = sizes(0)
nx = sizes(1)

vgridSizes = dimsizes(in_vgridFile->dz)
ndepth = vgridSizes(0)
depths = new(ndepth, double)

depths(0) = in_vgridFile->dz(0)
do k=1,ndepth-1
  depths(k) = in_vgridFile->dz(k) + depths(k-1)
end do 

; fill in kmt:
kmt = new((/ny,nx/), integer)
kmt = ndepth
kmt!0 = "ny"
kmt!1 = "nx"

in_maskFile = addfile(in_maskPath,"r")
msk = toint(in_maskFile->mask)

do j=0,ny-1
  do i=0,nx-1
    do k=0,ndepth-1
      if (in_topoFile->depth(j,i) .lt. depths(k))
        kmt(j,i) = k
        if ( msk(j,i).lt.1 .and. kmt(j,i).gt.0 ) then
          print(" "+j+" "+i+" "+in_topoFile->depth(j,i)+" "+depths(k)+" "+(k-1) )
        end if
        break    
      end if
    end do    
  end do
end do

dtr = get_d2r("double")
PI  = get_pi("double")

ulat = dtr * in_superGridFile->qlat(1:ny,1:nx)
ulon = dtr * in_superGridFile->qlon(1:ny,1:nx) 
htn = (in_superGridFile->dxCv) * 100.0 ; convert to cm
hte = (in_superGridFile->dyCu) * 100.0 ; convert to cm
hus = (in_superGridFile->dxCu) * 100.0 ; convert to cm
huw = (in_superGridFile->dyCv) * 100.0 ; convert to cm
angle = dtr * (in_superGridFile->angle)

print(" "+max(htn)+" "+min(htn))
print(" "+max(hte)+" "+min(hte))

; write the topography file
setfileoption("bin","WriteByteOrder","BigEndian")
system("/bin/rm -f " + out_topoPath)
fbindirwrite(out_topoPath, (kmt))

; write the horizontal grid file
system("/bin/rm -f " + out_hgridPath)
fbindirwrite(out_hgridPath, (ulat))
fbindirwrite(out_hgridPath, (ulon))
fbindirwrite(out_hgridPath, (htn))
fbindirwrite(out_hgridPath, (hte))
fbindirwrite(out_hgridPath, (hus))
fbindirwrite(out_hgridPath, (huw))
fbindirwrite(out_hgridPath, (angle))
```